In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from collections import Counter

In [2]:
file_path = "star_classification.csv"
stellar_df = pd.read_csv(file_path)
stellar_df.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543780e+18,GALAXY,0.634794,5812,56354,171
1,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176010e+19,GALAXY,0.779136,10445,58158,427
2,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030110e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891860e+18,GALAXY,0.116123,6121,56187,842


In [3]:
# Remove unecessary columns
cleaned_stellar_df = stellar_df.drop(columns=["obj_ID", "run_ID", "rerun_ID", "spec_obj_ID", "MJD"])
cleaned_stellar_df.head()

,alpha,delta,u,g,r,i,z,cam_col,field_ID,class,redshift,plate,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,2,79,GALAXY,0.634794,5812,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,5,119,GALAXY,0.779136,10445,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,2,120,GALAXY,0.644195,4576,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,3,214,GALAXY,0.932346,9149,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,3,137,GALAXY,0.116123,6121,842


In [4]:
x_cols = [i for i in cleaned_stellar_df.columns if i not in ('field_ID', 'fiber_ID', 'class')]
X = cleaned_stellar_df[x_cols]
y = cleaned_stellar_df['class']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'GALAXY': 14165, 'QSO': 14165, 'STAR': 14165})

In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\Users\john.molenda\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [9]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)



array([[7933, 1266, 5638],
       [ 190, 3877,  729],
       [1471,  445, 3451]], dtype=int64)

In [10]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6620207822000462

In [11]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.83      0.53      0.84      0.65      0.67      0.43     14837
        QSO       0.69      0.81      0.92      0.75      0.86      0.73      4796
       STAR       0.35      0.64      0.68      0.45      0.66      0.43      5367

avg / total       0.70      0.61      0.82      0.63      0.70      0.49     25000



In [12]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\Users\john.molenda\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [14]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.80      0.57      0.79      0.67      0.67      0.45     14837
        QSO       0.69      0.81      0.91      0.75      0.86      0.74      4796
       STAR       0.34      0.55      0.70      0.42      0.62      0.38      5367

avg / total       0.68      0.61      0.80      0.63      0.70      0.49     25000

